
# 필요한 라이브러리 설치 및 import 
1. Import and Install Dependencies

GPU를 사용하는 tensorflow와 mediapipe 설치 <br>

 mediapipe란 ? 주로 멀티 모델 오디오, 비디오 또는 모든 시계열 데이터를 구축하는 데 사용되는 프레임 워크입니다. <br>



In [1]:
# 필요한 라이브러리 설치 

#using tensorflow with GPU, mediapipe detacting movements
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# MP Holistic을 사용하여 이미지의 좌표 출력하기 

#mp_holistic = mp.solutions.holistic <br>
#mp.solutions.holistic 이란 ? -> 실시간 모션인식 (얼굴,몸의 분절을 실시간으로 추적한다.)

#참고 사이트 <br>
#https://google.github.io/mediapipe/solutions/holistic.html



#mp_drawing = 점을 연결하여 선을 보여준다.

In [2]:
mp_holistic = mp.solutions.holistic 
#mp.solutions.holistic 이란 ? -> 실시간 모션인식 (얼굴,몸의 분절을 실시간으로 추적한다.)

#Live perception of simultaneous human pose, face landmarks, 
#and hand tracking in real-time on mobile devices can enable various modern life applications
#https://google.github.io/mediapipe/solutions/holistic.html

mp_drawing = mp.solutions.drawing_utils # Drawing utilities
# 점을 연결하여 선을 보여준다.

In [3]:
# 이미지의 좌표를 추적하는 함수 만들기 
#이미지 -> RGB형식으로 만들어 모델에 대입한 결과를 results에 담고, 원상태로 만든 이미지와 results를 출력한다. 

def mediapipe_detection(image, model):   # 대상을 추적한다. 
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction 모델에 대입 
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results


In [5]:
print(results)
#<class 'mediapipe.python.solution_base.SolutionOutputs'>

results.face_landmarks.landmark
#x: 0.5144715905189514
#y: 0.24100039899349213
#z: -0.015107447281479836

#results -> <face, pose, left_hand, right_hand> 총 네 개의 결과값이 있다,
#하나의 값 (results.face_landmarks.landmark) 당 x,y,z 총 3개의 값이 3차원으로 존재한다. 

NameError: name 'results' is not defined

# 3. 위에서 얻은 results의 값을 numpy형태로 출력하기 
Extract Keypoint Values

results.pose_landmarks.landmark -> [res.x, res.y, res.z, res.visibility]으로 총 4개의 값으로 구성 <br>
그외는 <br>
results.face_landmarks.landmark -> [res.x, res.y, res.z]으로 구성 


In [4]:
# pose, face, lh, rh의 순서로 저장한다. 
# np.concatenate 를 하여 네개의 값을 하나의 넘파이 배열로 만든다.  
#만약 frame에 값이 출력되지 않았다면, (사진에서 왼손이 보이지 않아 left_hand의 result가 none으로 나올 경우) 0으로 채운다. 

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

#extract_keypoints(results).shape 
# (1662,) -> 33*4 + 468*3  + 21*3 + 21*3  

# 훈련데이터 동영상 넘파이 배열로 저장하기 
Convert Keypoint Values from videos

< 데이터 출처>  <br>
: Ai hub https://aihub.or.kr/keti_data_board/language_intelligence

<데이터 구성> <br>
데이터 크기 : 3000개의 수화 동영상 <br>
녹화 참여 인원수 : 총 10명 <br>
단어 종류 : 419개 

<데이터프레임 설명> <br>
dt : AI hub에서 가져온 동영상 파일 설명 엑셀파일 원본 <br>
dt1 : 문장과 단어가 모두 포함되어 있는 dt원본파일에서 단어 동영상만 추출 <br>
list_a : dt1의 한국어 컬럼을 단어의 개수 별로 정리한 데이터 프레임 <단어/횟수>로 구성.  

## 필요한 데이터프레임 생성 

In [7]:
#tensorflow에서 엑셀파일 여는 라이브러리 다운 

#!pip install pandas
#!pip install openpyxl
#!pip install fsspec

In [14]:
#dt 데이터프레임 생성    (419개 단어)
import pandas as pd
dt=pd.read_excel("D://suhua//10.09.xlsx")
dt.head()

,번호,언어 제공자 ID,취득연도,방향,타입(단어/문장),파일명,한국어,번호.1
0,1,1,2017,정면,단어,KETI_SL_0000000001.MOV,0,1
1,2,1,2017,정면,단어,KETI_SL_0000000002.MOV,1,2
2,3,1,2017,정면,단어,KETI_SL_0000000003.MOV,2,3
3,4,1,2017,정면,단어,KETI_SL_0000000004.MOV,3,4
4,5,1,2017,정면,단어,KETI_SL_0000000005.MOV,4,5


In [83]:
import pandas as pd
dtw=pd.read_excel("D://suhua//10.14.xlsx")
dtw.head()

,번호,언어 제공자 ID,취득연도,방향,타입(단어/문장),파일명,한국어,번호.1
0,1,1,2017,정면,백단어,KETI_SL_0000000001.MOV,0,1
1,2,1,2017,정면,백단어,KETI_SL_0000000002.MOV,1,2
2,3,1,2017,정면,백단어,KETI_SL_0000000003.MOV,2,3
3,4,1,2017,정면,백단어,KETI_SL_0000000004.MOV,3,4
4,5,1,2017,정면,백단어,KETI_SL_0000000005.MOV,4,5


In [84]:
# dtw한국어컬럼 중 float 형 int형으로 바꾸기 

for i in range(len(dtw['한국어'])):
    try:
        dtw['한국어'][i] = int(dtw['한국어'][i])
        dtw['한국어'][i] = str(dtw['한국어'][i])
    except:
        dtw['한국어'][i] == dtw['한국어'][i]

C:\Users\solag\AppData\Local\Temp/ipykernel_15932/2355230852.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtw['한국어'][i] = int(dtw['한국어'][i])
C:\Users\solag\AppData\Local\Temp/ipykernel_15932/2355230852.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtw['한국어'][i] = str(dtw['한국어'][i])


In [85]:
#파일명 마지막 세글자.avi로 바꾸기 
dtw['파일명'] = dtw['파일명'].apply(lambda x:x[:-3]) + dtw['파일명'].apply(lambda x:x[-3:]).replace('MOV','avi')
dtw['파일명']

0       KETI_SL_0000000001.avi
1       KETI_SL_0000000002.avi
2       KETI_SL_0000000003.avi
3       KETI_SL_0000000004.avi
4       KETI_SL_0000000005.avi
                 ...          
4699    KETI_SL_0000010371.avi
4700    KETI_SL_0000010372.avi
4701    KETI_SL_0000010373.avi
4702    KETI_SL_0000010374.avi
4703    KETI_SL_0000010375.avi
Name: 파일명, Length: 4704, dtype: object

In [86]:
## dt1 데이터 프레임 생성 
#전체 데이터 단어만 선택한 후 3000개 데이터 선택 
import pandas as pd
dtw1=dtw[dtw['타입(단어/문장)']=='백단어']
dtw1 = dtw1[:3000]
dtw1['한국어']

0        0
1        1
2        2
3        3
4        4
        ..
2927     6
2928     7
2929     8
2930     9
2931    10
Name: 한국어, Length: 703, dtype: object

In [88]:
# 단어별로 동영상이 몇개 있는지 출력하는 list_a 데이터 프레임 생성 

#한국어컬럼만 추출
aaw = dtw1[['한국어']].copy()

#한국어 컬럼에 대한 개수 추출
list_aw =aaw['한국어'].value_counts()

#인덱스를 없애서 총 2개의 컬럼을 가진 데이터 프레임 생성 [한국어.횟수]
list_aw =list_aw.reset_index()

# 데이터프레임 컬렴명 변겅 
list_aw.columns = ['한국어','횟수']
list_aw = list_aw.sort_values('한국어')
list_aw.head()
#list_aw.tail()

#len(list_aw) #329


,한국어,횟수
0,0,8
2,1,8
6,10,8
73,1000,7
72,10000,7


In [15]:
# dt 한국어컬럼 중 float 형 int형으로 바꾸기 

for i in range(len(dt['한국어'])):
    try:
        dt['한국어'][i] = int(dt['한국어'][i])
        dt['한국어'][i] = str(dt['한국어'][i])
    except:
        dt['한국어'][i] == dt['한국어'][i]

C:\Users\solag\AppData\Local\Temp/ipykernel_15932/2159450155.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt['한국어'][i] = int(dt['한국어'][i])
C:\Users\solag\AppData\Local\Temp/ipykernel_15932/2159450155.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt['한국어'][i] = str(dt['한국어'][i])


In [16]:
#파일명 마지막 세글자.avi로 바꾸기 
dt['파일명'] = dt['파일명'].apply(lambda x:x[:-3]) + dt['파일명'].apply(lambda x:x[-3:]).replace('MOV','avi')
dt['파일명']

0       KETI_SL_0000000001.avi
1       KETI_SL_0000000002.avi
2       KETI_SL_0000000003.avi
3       KETI_SL_0000000004.avi
4       KETI_SL_0000000005.avi
                 ...          
4699    KETI_SL_0000010371.avi
4700    KETI_SL_0000010372.avi
4701    KETI_SL_0000010373.avi
4702    KETI_SL_0000010374.avi
4703    KETI_SL_0000010375.avi
Name: 파일명, Length: 4704, dtype: object

In [17]:
## dt1 데이터 프레임 생성 
#전체 데이터 단어만 선택한 후 3000개 데이터 선택 
import pandas as pd
dt1=dt[dt['타입(단어/문장)']=='단어']
dt1 = dt1[:3000]

In [18]:
dt1['한국어']

0        0
1        1
2        2
3        3
4        4
        ..
2995    74
2996    75
2997    76
2998    77
2999    78
Name: 한국어, Length: 3000, dtype: object

In [19]:
# 단어별로 동영상이 몇개 있는지 출력하는 list_a 데이터 프레임 생성 

#한국어컬럼만 추출
aa = dt1[['한국어']].copy()

#한국어 컬럼에 대한 개수 추출
list_a =aa['한국어'].value_counts()

#인덱스를 없애서 총 2개의 컬럼을 가진 데이터 프레임 생성 [한국어.횟수]
list_a =list_a.reset_index()

# 데이터프레임 컬렴명 변겅 
list_a.columns = ['한국어','횟수']
list_a = list_a.sort_values('한국어')
list_a.head()

#len(list_a)
#419

,한국어,횟수
0,0,8
22,1,8
49,10,8
328,100,7
325,1000,7


## numpy형태로 변환한 파일 담을 폴더 생성 

<폴더 구조> <br>
train 폴더 <br>
단어의 뜻을 이름으로 하는 폴더 419개  <br>
각각의 단어 폴더 안에 단어 당 촬영한 동영상 개수 만큼 폴더 생성   <br>
각각의 동영상 개수 폴더 안에 프레임 횟수만큼 폴더를 만들고 하나의 폴더안에 프레임 하나의 넘파이 배열이 담긴 파이썬 파일 첨부 

In [21]:
# 단어별 -동영상개수만큼의 폴더 생성하기 
import os
# train 폴더 생성
os.mkdir( 'c://Users//User//sign//train')

# 단어별 폴더 생성 
for k in range(419): 
    a = 'C://Users//User//sign//train//' + str(list_a['한국어'][k])  # 동영상 이름 
    os.mkdir(a)
    # 단어 폴더 안에 동영상의 횟수만큼 폴더 생성 
    for i in range(list_a['횟수'][k]):
        b= 'C://Users//User//sign//train//' + str(list_a['한국어'][k]) + '//' + str(i)
        os.mkdir(b)

## video3000에 있는 동영상 읽어와서 numpy배열로 변환한 후 위에서 만든 train 폴더에 저장 


In [26]:
#폴더의 개수 출력하는 함수 생성 
def dir_empty(dir_path):
    return len(os.listdir(dir_path))



# holistic 모델 사용 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # 3000개의 동영상 분석 
    for k in range(1000, 3001):
        # dt1['파일명']의 동영상 3000개 분석 
        #dt1 -> 단어별로 정리한 동영상 파일 3000개 
        
        videos = "C://Users//User//sign//video3000//" + dt1['파일명'][k] 

        
        #openCV를 열고 동영상을 읽어들인다. -> 동영상의 처음부터 끝까지 frame형태로 기록된다. 
        # -> 출력물 ret. frame 
        
        # ret이란 frame이 존재하는지 존재하지 않는지 True, False로 출력한다. 
        # 동영상이 끝나 더이상의 frame이 존재하지 않으면 False출력 
        
        cam = cv2.VideoCapture(videos)
        
        #출력하는 동영상의 해당 단어 kor에 담기 
        kor = dt1['한국어'][k]
        
        # videocount -> 해당 단어의 몇번째 동영상인지 출력 
        videocount =0

      # 
        while (True):
            #만약 해당 단어 폴더의 vidoecount의 이름을 가진 폴더의 길이가 0이라면(비어있다면)
            #  해당단어 - video count - frame 개수 별로 만든 폴더에 넘파이 파일 저장 
            if dir_empty('C://Users//User//sign//train//' + kor + '//'+str(videocount)) == 0:
                
                #프레임 개수를 알려주는 currentframe 
                currentframe = 0

                while (True):
                    # 여기서의 True 가 밑의 ret == True로 연결
                    #ret 이 True 라면 계속 반복, 즉 프레임이 남아있다면 계속해서 읽어들인다. 



                    # 위에서 읽은 cam을 frame별로 하나씩 읽어들이겠다.
                    ret,frame = cam.read()


                    # 만약 프레임이 남아있다면 
                    if ret == True:
                        
                        image, results = mediapipe_detection(frame, holistic)  # frame에 있는 점들을 추적하여 result에 넣는다. 
                        keypoints = extract_keypoints(results) #result를 하나의 넘파이 배열로 만든다
                        
                        # 프레임 번호를 폴더명으로 하는 폴더 생성 
                        os.mkdir('C://Users//User//sign//train//' + kor + '//'+str(videocount)+'//'+ str(currentframe))
                        
                        #위에서 만든 폴더에 넘파이 배열 파일을 넣겠다. 
                        npy_path = 'C://Users//User//sign//train//' + kor + '//'+str(videocount)+'//'+ str(currentframe)+'//' +str(currentframe)
                        np.save(npy_path, keypoints)
                        
                        #다음프레임으로 이동 
                        currentframe +=1
                        


                    #만약 프레임이 끝났다면 즉, 하나의 동영상이 완료되었다면 종료 (두번째 while 문 종료)
                    else:
                        break
                
                # 하나의 동영상 추출 완료 ( if문 종료 )
                # dir_empty('C://Users//User//sign//train//' + kor + '//'+str(videocount)) == 0: 종료 
                # 완료를 알려주는 문장 출력 
                print(kor + '단어' + str(currentframe) + '프레임 추출 완료======================================')
                
                break
                
    #첫번째 while 문 
    #만약 ir_empty('C://Users//User//sign//train//' + kor + '//'+str(videocount)) == 0 이 아니라면, 
    # 즉 단어 - 동영상 횟수를 나타내는 폴더가 비어있지 않다면 다음폴더로 이동 
    #ex ) 사자- 1번째 동영상의 폴더의 길이가 0이아니라면 사자-2 번째 폴더로 이동하고 위의 문장을 다시 실행 
    
            else:
                videocount +=1
                continue
        
        

낫단어103프레임 추출 완료======================================
낯선남자단어72프레임 추출 완료======================================
낯선사람단어80프레임 추출 완료======================================
낯선여자단어84프레임 추출 완료======================================
내년단어69프레임 추출 완료======================================
내일단어136프레임 추출 완료======================================
냄새나다단어136프레임 추출 완료======================================
노원구단어74프레임 추출 완료======================================
논단어86프레임 추출 완료======================================
농약단어99프레임 추출 완료======================================
누나단어136프레임 추출 완료======================================
누수단어92프레임 추출 완료======================================
누전단어81프레임 추출 완료======================================
누출단어88프레임 추출 완료======================================
눈단어78프레임 추출 완료======================================
다리단어72프레임 추출 완료======================================
다음단어136프레임 추출 완료======================================
달(월)단어151프레임 추출 완료======================================
대문앞단어107프레임 추출

옆쪽단어131프레임 추출 완료======================================
오늘단어58프레임 추출 완료======================================
오른쪽단어69프레임 추출 완료======================================
오른쪽-귀단어121프레임 추출 완료======================================
오른쪽-눈단어132프레임 추출 완료======================================
오빠단어120프레임 추출 완료======================================
옥상단어84프레임 추출 완료======================================
올해단어71프레임 추출 완료======================================
왼쪽단어73프레임 추출 완료======================================
왼쪽-귀단어121프레임 추출 완료======================================
왼쪽-눈단어121프레임 추출 완료======================================
욕실단어80프레임 추출 완료======================================
용산구단어85프레임 추출 완료======================================
우리집단어68프레임 추출 완료======================================
운동장단어81프레임 추출 완료======================================
월요일단어121프레임 추출 완료======================================
위단어107프레임 추출 완료======================================
위에단어71프레임 추출 완료======================================
위협단어78

42단어71프레임 추출 완료======================================
43단어128프레임 추출 완료======================================
44단어120프레임 추출 완료======================================
45단어136프레임 추출 완료======================================
46단어123프레임 추출 완료======================================
47단어128프레임 추출 완료======================================
48단어78프레임 추출 완료======================================
49단어113프레임 추출 완료======================================
50단어119프레임 추출 완료======================================
51단어128프레임 추출 완료======================================
52단어187프레임 추출 완료======================================
53단어128프레임 추출 완료======================================
54단어71프레임 추출 완료======================================
55단어115프레임 추출 완료======================================
56단어117프레임 추출 완료======================================
57단어137프레임 추출 완료======================================
58단어137프레임 추출 완료======================================
59단어126프레임 추출 완료======================================
60단어80프레임 추출 

뜨거운물단어81프레임 추출 완료======================================
마당단어111프레임 추출 완료======================================
마포구단어88프레임 추출 완료======================================
말려주세요단어88프레임 추출 완료======================================
말벌단어115프레임 추출 완료======================================
맹견단어105프레임 추출 완료======================================
머리단어87프레임 추출 완료======================================
멧돼지단어92프레임 추출 완료======================================
목단어121프레임 추출 완료======================================
목요일단어72프레임 추출 완료======================================
무너지다단어60프레임 추출 완료======================================
무릎단어70프레임 추출 완료======================================
문틈단어113프레임 추출 완료======================================
물단어141프레임 추출 완료======================================
밑에단어67프레임 추출 완료======================================
바다단어91프레임 추출 완료======================================
반점생기다단어116프레임 추출 완료======================================
발단어56프레임 추출 완료======================================
발가락단어110프레임

작년단어82프레임 추출 완료======================================
작은방단어91프레임 추출 완료======================================
장난감단어79프레임 추출 완료======================================
장단지단어96프레임 추출 완료======================================
절단단어77프레임 추출 완료======================================
제초제단어116프레임 추출 완료======================================
조난단어87프레임 추출 완료======================================
종로구단어79프레임 추출 완료======================================
주단어136프레임 추출 완료======================================
중구단어86프레임 추출 완료======================================
중랑구단어84프레임 추출 완료======================================
지난단어170프레임 추출 완료======================================
지혈대단어130프레임 추출 완료======================================
진통제단어116프레임 추출 완료======================================
질식단어86프레임 추출 완료======================================
집단어64프레임 추출 완료======================================
집단폭행단어111프레임 추출 완료======================================
차밖단어90프레임 추출 완료======================================
차안단어92프레임 추출 완

...

협박단어208프레임 추출 완료======================================
형단어166프레임 추출 완료======================================
호흡곤란단어196프레임 추출 완료======================================
호흡기단어115프레임 추출 완료======================================
홍수단어126프레임 추출 완료======================================
화상단어114프레임 추출 완료======================================
화약단어107프레임 추출 완료======================================
화요일단어186프레임 추출 완료======================================
화장실단어201프레임 추출 완료======================================
화재단어110프레임 추출 완료======================================
장단지단어127프레임 추출 완료======================================
0단어197프레임 추출 완료======================================
1단어171프레임 추출 완료======================================
2단어171프레임 추출 완료======================================
3단어149프레임 추출 완료======================================
4단어191프레임 추출 완료======================================
5단어178프레임 추출 완료======================================
6단어167프레임 추출 완료======================================
7단어156프레임 추출

공장단어166프레임 추출 완료======================================
관악구단어153프레임 추출 완료======================================
광진구단어161프레임 추출 완료======================================
교통사고단어182프레임 추출 완료======================================
구급대단어110프레임 추출 완료======================================
구급대원단어185프레임 추출 완료======================================
구급차단어97프레임 추출 완료======================================
구로구단어166프레임 추출 완료======================================
구청단어70프레임 추출 완료======================================
구해주세요단어167프레임 추출 완료======================================
귀단어63프레임 추출 완료======================================
금가다단어61프레임 추출 완료======================================
금요일단어196프레임 추출 완료======================================
금천구단어82프레임 추출 완료======================================
급류단어95프레임 추출 완료======================================
기절단어71프레임 추출 완료======================================
기절하다단어71프레임 추출 완료======================================
깔리다단어66프레임 추출 완료======================================
끓는

약국단어66프레임 추출 완료======================================
약사단어68프레임 추출 완료======================================
양천구단어94프레임 추출 완료======================================
어깨단어61프레임 추출 완료======================================
어제단어53프레임 추출 완료======================================
어지러움단어66프레임 추출 완료======================================
언니단어50프레임 추출 완료======================================
얼굴단어66프레임 추출 완료======================================
엄마단어60프레임 추출 완료======================================
엘리베이터단어66프레임 추출 완료======================================
여자친구단어82프레임 추출 완료======================================
여학생단어181프레임 추출 완료======================================
연기단어77프레임 추출 완료======================================
연락해주세요단어89프레임 추출 완료======================================
열단어53프레임 추출 완료======================================
열나다단어69프레임 추출 완료======================================
열어주세요단어86프레임 추출 완료======================================
엽총단어158프레임 추출 완료======================================
영등포구단어81프레

19단어174프레임 추출 완료======================================
20단어77프레임 추출 완료======================================
21단어143프레임 추출 완료======================================
22단어154프레임 추출 완료======================================
23단어193프레임 추출 완료======================================
24단어197프레임 추출 완료======================================
25단어192프레임 추출 완료======================================
26단어198프레임 추출 완료======================================
27단어88프레임 추출 완료======================================
28단어77프레임 추출 완료======================================
29단어202프레임 추출 완료======================================
30단어174프레임 추출 완료======================================
31단어81프레임 추출 완료======================================
32단어207프레임 추출 완료======================================
33단어86프레임 추출 완료======================================
34단어191프레임 추출 완료======================================
35단어179프레임 추출 완료======================================
36단어194프레임 추출 완료======================================
37단어197프레임 추출 완

KeyError: 3000

In [ ]:
## 3차원 형태의 넘파이 배열을 모델에 넣기 위해서는 모든 프레임의 개수가 같아야 한다.
# 프레임의 개수가 가장 많은 영상을 찾아 그 영상에 맞춰 모든 영상의 마지막 프레임을 해당 프레임의 개수만큼 복사한다. 

In [6]:
import os
import shutil
file_list_1=os.listdir("E://Users//User//sign//test")

print(file_list_1)
print(file_list_1[1])
print(len(file_list_1))

['0', '0_2', '93', '수요일']
0_2
4


In [43]:
import os
import shutil
num=[]
cnt=0
file_list_1=os.listdir("E://Users//User//sign//test")
for i in range(len(file_list_1)):
    path="E://Users//User//sign//test//" + str(file_list_1[i])
    #print(path)
    file_list_2=os.listdir(path)
    for j in range(len(file_list_2)):
        path2="E://Users//User//sign//test//" + str(file_list_1[i])+'//'+str(file_list_2[j])
        #print(path2)
        file_list_3=os.listdir(path2)
        for k in range(len(file_list_3)):
            path3="E://Users//User//sign//test//" + str(file_list_1[i])+'//'+str(file_list_2[j])+'//'+str(file_list_3[k])
            #print(path3)
            file_list_4=os.listdir(path3)
            for h in file_list_4:
                print(path3,h) #모든 파일 명 추출                 

E://Users//User//sign//test//0//0//0 0.npy
E://Users//User//sign//test//0//0//1 1.npy
E://Users//User//sign//test//0//0//10 10.npy
E://Users//User//sign//test//0//0//100 100.npy
E://Users//User//sign//test//0//0//101 101.npy
E://Users//User//sign//test//0//0//102 102.npy
E://Users//User//sign//test//0//0//103 103.npy
E://Users//User//sign//test//0//0//104 104.npy
E://Users//User//sign//test//0//0//105 105.npy
E://Users//User//sign//test//0//0//106 106.npy
E://Users//User//sign//test//0//0//107 107.npy
E://Users//User//sign//test//0//0//108 108.npy
E://Users//User//sign//test//0//0//109 109.npy
E://Users//User//sign//test//0//0//11 11.npy
E://Users//User//sign//test//0//0//110 110.npy
E://Users//User//sign//test//0//0//111 111.npy
E://Users//User//sign//test//0//0//112 112.npy
E://Users//User//sign//test//0//0//113 113.npy
E://Users//User//sign//test//0//0//114 114.npy
E://Users//User//sign//test//0//0//115 115.npy
E://Users//User//sign//test//0//0//116 116.npy
E://Users//User//sign//te

E://Users//User//sign//test//0//3//118 118.npy
E://Users//User//sign//test//0//3//119 119.npy
E://Users//User//sign//test//0//3//12 12.npy
E://Users//User//sign//test//0//3//120 119.npy
E://Users//User//sign//test//0//3//121 119.npy
E://Users//User//sign//test//0//3//122 119.npy
E://Users//User//sign//test//0//3//123 119.npy
E://Users//User//sign//test//0//3//124 119.npy
E://Users//User//sign//test//0//3//125 119.npy
E://Users//User//sign//test//0//3//126 119.npy
E://Users//User//sign//test//0//3//127 119.npy
E://Users//User//sign//test//0//3//128 119.npy
E://Users//User//sign//test//0//3//129 119.npy
E://Users//User//sign//test//0//3//13 13.npy
E://Users//User//sign//test//0//3//130 119.npy
E://Users//User//sign//test//0//3//131 119.npy
E://Users//User//sign//test//0//3//132 119.npy
E://Users//User//sign//test//0//3//133 119.npy
E://Users//User//sign//test//0//3//134 119.npy
E://Users//User//sign//test//0//3//135 119.npy
E://Users//User//sign//test//0//3//136 119.npy
E://Users//User//

E://Users//User//sign//test//0//6//108 108.npy
E://Users//User//sign//test//0//6//109 109.npy
E://Users//User//sign//test//0//6//11 11.npy
E://Users//User//sign//test//0//6//110 110.npy
E://Users//User//sign//test//0//6//111 111.npy
E://Users//User//sign//test//0//6//112 112.npy
E://Users//User//sign//test//0//6//113 113.npy
E://Users//User//sign//test//0//6//114 114.npy
E://Users//User//sign//test//0//6//115 115.npy
E://Users//User//sign//test//0//6//116 116.npy
E://Users//User//sign//test//0//6//117 117.npy
E://Users//User//sign//test//0//6//118 118.npy
E://Users//User//sign//test//0//6//119 119.npy
E://Users//User//sign//test//0//6//12 12.npy
E://Users//User//sign//test//0//6//120 120.npy
E://Users//User//sign//test//0//6//121 121.npy
E://Users//User//sign//test//0//6//122 122.npy
E://Users//User//sign//test//0//6//123 123.npy
E://Users//User//sign//test//0//6//124 124.npy
E://Users//User//sign//test//0//6//125 125.npy
E://Users//User//sign//test//0//6//126 126.npy
E://Users//User//

E://Users//User//sign//test//0_2//1//0 0.npy
E://Users//User//sign//test//0_2//1//1 1.npy
E://Users//User//sign//test//0_2//1//10 10.npy
E://Users//User//sign//test//0_2//1//100 100.npy
E://Users//User//sign//test//0_2//1//101 101.npy
E://Users//User//sign//test//0_2//1//102 102.npy
E://Users//User//sign//test//0_2//1//103 103.npy
E://Users//User//sign//test//0_2//1//104 104.npy
E://Users//User//sign//test//0_2//1//105 105.npy
E://Users//User//sign//test//0_2//1//106 106.npy
E://Users//User//sign//test//0_2//1//107 107.npy
E://Users//User//sign//test//0_2//1//108 108.npy
E://Users//User//sign//test//0_2//1//109 109.npy
E://Users//User//sign//test//0_2//1//11 11.npy
E://Users//User//sign//test//0_2//1//110 110.npy
E://Users//User//sign//test//0_2//1//111 111.npy
E://Users//User//sign//test//0_2//1//112 112.npy
E://Users//User//sign//test//0_2//1//113 113.npy
E://Users//User//sign//test//0_2//1//114 114.npy
E://Users//User//sign//test//0_2//1//115 115.npy
E://Users//User//sign//test//0_2

E://Users//User//sign//test//0_2//4//29 29.npy
E://Users//User//sign//test//0_2//4//290 190.npy
E://Users//User//sign//test//0_2//4//291 190.npy
E://Users//User//sign//test//0_2//4//292 190.npy
E://Users//User//sign//test//0_2//4//293 190.npy
E://Users//User//sign//test//0_2//4//294 190.npy
E://Users//User//sign//test//0_2//4//295 190.npy
E://Users//User//sign//test//0_2//4//296 190.npy
E://Users//User//sign//test//0_2//4//297 190.npy
E://Users//User//sign//test//0_2//4//298 190.npy
E://Users//User//sign//test//0_2//4//299 190.npy
E://Users//User//sign//test//0_2//4//3 3.npy
E://Users//User//sign//test//0_2//4//30 30.npy
E://Users//User//sign//test//0_2//4//300 190.npy
E://Users//User//sign//test//0_2//4//301 190.npy
E://Users//User//sign//test//0_2//4//302 190.npy
E://Users//User//sign//test//0_2//4//303 190.npy
E://Users//User//sign//test//0_2//4//304 190.npy
E://Users//User//sign//test//0_2//4//305 190.npy
E://Users//User//sign//test//0_2//4//306 190.npy
E://Users//User//sign//test/

E://Users//User//sign//test//0_2//7//308 202.npy
E://Users//User//sign//test//0_2//7//309 202.npy
E://Users//User//sign//test//0_2//7//31 31.npy
E://Users//User//sign//test//0_2//7//310 202.npy
E://Users//User//sign//test//0_2//7//311 202.npy
E://Users//User//sign//test//0_2//7//312 202.npy
E://Users//User//sign//test//0_2//7//313 202.npy
E://Users//User//sign//test//0_2//7//314 202.npy
E://Users//User//sign//test//0_2//7//315 202.npy
E://Users//User//sign//test//0_2//7//316 202.npy
E://Users//User//sign//test//0_2//7//317 202.npy
E://Users//User//sign//test//0_2//7//318 202.npy
E://Users//User//sign//test//0_2//7//319 202.npy
E://Users//User//sign//test//0_2//7//32 32.npy
E://Users//User//sign//test//0_2//7//320 202.npy
E://Users//User//sign//test//0_2//7//321 202.npy
E://Users//User//sign//test//0_2//7//322 202.npy
E://Users//User//sign//test//0_2//7//323 202.npy
E://Users//User//sign//test//0_2//7//324 202.npy
E://Users//User//sign//test//0_2//7//325 202.npy
E://Users//User//sign//t

E://Users//User//sign//test//93//2//217 165.npy
E://Users//User//sign//test//93//2//218 165.npy
E://Users//User//sign//test//93//2//219 165.npy
E://Users//User//sign//test//93//2//22 22.npy
E://Users//User//sign//test//93//2//220 165.npy
E://Users//User//sign//test//93//2//221 165.npy
E://Users//User//sign//test//93//2//222 165.npy
E://Users//User//sign//test//93//2//223 165.npy
E://Users//User//sign//test//93//2//224 165.npy
E://Users//User//sign//test//93//2//225 165.npy
E://Users//User//sign//test//93//2//226 165.npy
E://Users//User//sign//test//93//2//227 165.npy
E://Users//User//sign//test//93//2//228 165.npy
E://Users//User//sign//test//93//2//229 165.npy
E://Users//User//sign//test//93//2//23 23.npy
E://Users//User//sign//test//93//2//230 165.npy
E://Users//User//sign//test//93//2//231 165.npy
E://Users//User//sign//test//93//2//232 165.npy
E://Users//User//sign//test//93//2//233 165.npy
E://Users//User//sign//test//93//2//234 165.npy
E://Users//User//sign//test//93//2//235 165.

E://Users//User//sign//test//93//5//271 93.npy
E://Users//User//sign//test//93//5//272 93.npy
E://Users//User//sign//test//93//5//273 93.npy
E://Users//User//sign//test//93//5//274 93.npy
E://Users//User//sign//test//93//5//275 93.npy
E://Users//User//sign//test//93//5//276 93.npy
E://Users//User//sign//test//93//5//277 93.npy
E://Users//User//sign//test//93//5//278 93.npy
E://Users//User//sign//test//93//5//279 93.npy
E://Users//User//sign//test//93//5//28 28.npy
E://Users//User//sign//test//93//5//280 93.npy
E://Users//User//sign//test//93//5//281 93.npy
E://Users//User//sign//test//93//5//282 93.npy
E://Users//User//sign//test//93//5//283 93.npy
E://Users//User//sign//test//93//5//284 93.npy
E://Users//User//sign//test//93//5//285 93.npy
E://Users//User//sign//test//93//5//286 93.npy
E://Users//User//sign//test//93//5//287 93.npy
E://Users//User//sign//test//93//5//288 93.npy
E://Users//User//sign//test//93//5//289 93.npy
E://Users//User//sign//test//93//5//29 29.npy
E://Users//User

E://Users//User//sign//test//수요일//1//284 49.npy
E://Users//User//sign//test//수요일//1//285 49.npy
E://Users//User//sign//test//수요일//1//286 49.npy
E://Users//User//sign//test//수요일//1//287 49.npy
E://Users//User//sign//test//수요일//1//288 49.npy
E://Users//User//sign//test//수요일//1//289 49.npy
E://Users//User//sign//test//수요일//1//29 29.npy
E://Users//User//sign//test//수요일//1//290 49.npy
E://Users//User//sign//test//수요일//1//291 49.npy
E://Users//User//sign//test//수요일//1//292 49.npy
E://Users//User//sign//test//수요일//1//293 49.npy
E://Users//User//sign//test//수요일//1//294 49.npy
E://Users//User//sign//test//수요일//1//295 49.npy
E://Users//User//sign//test//수요일//1//296 49.npy
E://Users//User//sign//test//수요일//1//297 49.npy
E://Users//User//sign//test//수요일//1//298 49.npy
E://Users//User//sign//test//수요일//1//299 49.npy
E://Users//User//sign//test//수요일//1//3 3.npy
E://Users//User//sign//test//수요일//1//30 30.npy
E://Users//User//sign//test//수요일//1//300 49.npy
E://Users//User//sign//test//수요일//1//301 49.n

E://Users//User//sign//test//수요일//4//62 62.npy
E://Users//User//sign//test//수요일//4//63 63.npy
E://Users//User//sign//test//수요일//4//64 64.npy
E://Users//User//sign//test//수요일//4//65 65.npy
E://Users//User//sign//test//수요일//4//66 66.npy
E://Users//User//sign//test//수요일//4//67 67.npy
E://Users//User//sign//test//수요일//4//68 68.npy
E://Users//User//sign//test//수요일//4//69 69.npy
E://Users//User//sign//test//수요일//4//7 7.npy
E://Users//User//sign//test//수요일//4//70 70.npy
E://Users//User//sign//test//수요일//4//71 71.npy
E://Users//User//sign//test//수요일//4//72 72.npy
E://Users//User//sign//test//수요일//4//73 73.npy
E://Users//User//sign//test//수요일//4//74 74.npy
E://Users//User//sign//test//수요일//4//75 75.npy
E://Users//User//sign//test//수요일//4//76 76.npy
E://Users//User//sign//test//수요일//4//77 77.npy
E://Users//User//sign//test//수요일//4//78 78.npy
E://Users//User//sign//test//수요일//4//79 79.npy
E://Users//User//sign//test//수요일//4//8 8.npy
E://Users//User//sign//test//수요일//4//80 80.npy
E://Users//User//

In [11]:
##최종
for i in range(len(file_list_1)):
    path="E://Users//User//sign//test//" + str(file_list_1[i])
    file_list_2=os.listdir(path)
    for p in range(len(file_list_2)):
        path2=path+'//'+str(file_list_2[p])
        file_list_3=os.listdir(path2)
        #print(path2, len(file_list_3))
        path3=path2+'//'+str(len(file_list_3)-1)
        
        num=len(file_list_3)-1
        
        if num < 432 :
            
            while (True):
                if num < 432:
                    num=num+1
                    path4=path2+'//'+str(num)
                    print(path4)

                    if num < 432:
                        shutil.copytree(path3, path4)
                        #print(path4)

                        continue

                    #print(path4)
                    break
                
                #shutil.copytree("C://Users//User//sign//test//0", "C://Users//User//sign//test//0_2")
        
        
#print(max(count_list3))     0~431   432개


#len(file_list_3)

E://Users//User//sign//test//0//0//432
E://Users//User//sign//test//0//1//432
E://Users//User//sign//test//0//2//432
E://Users//User//sign//test//0//3//432
E://Users//User//sign//test//0//4//432
E://Users//User//sign//test//0//5//432
E://Users//User//sign//test//0//6//432
E://Users//User//sign//test//0//7//432
E://Users//User//sign//test//0_2//0//432
E://Users//User//sign//test//0_2//1//432
E://Users//User//sign//test//0_2//2//432
E://Users//User//sign//test//0_2//3//432
E://Users//User//sign//test//0_2//4//432
E://Users//User//sign//test//0_2//5//432
E://Users//User//sign//test//0_2//6//432
E://Users//User//sign//test//0_2//7//432
E://Users//User//sign//test//93//0//432
E://Users//User//sign//test//93//1//432
E://Users//User//sign//test//93//2//432
E://Users//User//sign//test//93//3//432
E://Users//User//sign//test//93//4//432
E://Users//User//sign//test//93//5//432
E://Users//User//sign//test//93//6//432
E://Users//User//sign//test//수요일//0//432
E://Users//User//sign//test//수요일//1//43

# 6. 데이터 라벨링하기 


### 동영상 데이터 라벨링하기


In [ ]:
#!pip uninstall protobuf
#!pip install --no-binary=protobuf protobuf

In [20]:
# 다른 액션에 대한 레이블 만들기 
# 필요한 데이터 임포트
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [21]:
# 정답데이터 라벨링 하기 
# dictionary형태의 {한국단어 : 단어에 대한 숫자}

label_map = {label:num for num, label in enumerate(list_a['한국어'])} # enumerate-> 0 부터 순서대로 나타내 주는 것 

In [19]:
label_map

{'0': 2921,
 '1': 2922,
 '2': 2923,
 '3': 2924,
 '4': 2925,
 '5': 2926,
 '6': 2927,
 '7': 2928,
 '8': 2929,
 '9': 2930,
 '10': 2931,
 '11': 2932,
 '12': 2933,
 '13': 2934,
 '14': 2935,
 '15': 2936,
 '16': 2937,
 '17': 2938,
 '18': 2939,
 '19': 2940,
 '20': 2941,
 '21': 2942,
 '22': 2943,
 '23': 2944,
 '24': 2945,
 '25': 2946,
 '26': 2947,
 '27': 2948,
 '28': 2949,
 '29': 2950,
 '30': 2951,
 '31': 2952,
 '32': 2953,
 '33': 2954,
 '34': 2955,
 '35': 2956,
 '36': 2957,
 '37': 2958,
 '38': 2959,
 '39': 2960,
 '40': 2961,
 '41': 2962,
 '42': 2963,
 '43': 2964,
 '44': 2965,
 '45': 2966,
 '46': 2967,
 '47': 2968,
 '48': 2969,
 '49': 2970,
 '50': 2971,
 '51': 2972,
 '52': 2973,
 '53': 2974,
 '54': 2975,
 '55': 2976,
 '56': 2977,
 '57': 2978,
 '58': 2979,
 '59': 2980,
 '60': 2981,
 '61': 2982,
 '62': 2983,
 '63': 2984,
 '64': 2985,
 '65': 2986,
 '66': 2987,
 '67': 2988,
 '68': 2989,
 '69': 2990,
 '70': 2991,
 '71': 2992,
 '72': 2993,
 '73': 2994,
 '74': 2995,
 '75': 2996,
 '76': 2997,
 '77': 29

In [106]:
label_map = {label:num for num, label in enumerate(list_aw['한국어'])}
label_map    #100

{'0': 0,
 '1': 1,
 '10': 2,
 '1000': 3,
 '10000': 4,
 '112': 5,
 '119': 6,
 '2': 7,
 '3': 8,
 '4': 9,
 '5': 10,
 '6': 11,
 '7': 12,
 '8': 13,
 '9': 14,
 '구해주세요': 15,
 '반점생기다': 16,
 '배고프다': 17,
 '병원': 18,
 '보내주세요(경찰)': 19,
 '보내주세요(구급차)': 20,
 '불나다': 21,
 '비상약': 22,
 '뼈': 23,
 '살충제': 24,
 '살해': 25,
 '삼키다': 26,
 '선생님': 27,
 '성폭행': 28,
 '소방차': 29,
 '소화기': 30,
 '손가락': 31,
 '손목': 32,
 '술취한 사람': 33,
 '숨을안쉬다': 34,
 '신고하세요(경찰)': 35,
 '심장마비': 36,
 '아파트': 37,
 '알려주세요': 38,
 '약국': 39,
 '어깨': 40,
 '어린이': 41,
 '어제': 42,
 '어지러움': 43,
 '얼굴': 44,
 '엄마': 45,
 '엘리베이터': 46,
 '연락해주세요': 47,
 '열나다': 48,
 '열어주세요': 49,
 '옆집사람': 50,
 '욕실': 51,
 '운동장': 52,
 '위에': 53,
 '유리': 54,
 '음식물': 55,
 '응급대원': 56,
 '응급처리': 57,
 '의사': 58,
 '이마': 59,
 '이물질': 60,
 '이상한사람': 61,
 '임산부': 62,
 '자동차': 63,
 '자살': 64,
 '절단': 65,
 '조난': 66,
 '지혈대': 67,
 '진통제': 68,
 '집': 69,
 '집단폭행': 70,
 '창백하다': 71,
 '체온계': 72,
 '총': 73,
 '추락': 74,
 '출산': 75,
 '출혈': 76,
 '칼': 77,
 '코': 78,
 '택시': 79,
 '토하다': 80,
 '통학버스': 81,
 '트럭': 82,
 '팔': 83,
 '팔꿈치

In [115]:
DATA_PATH  =  'C:\\data\\train2\\'
for kor in list_a['한국어']:
    #print(kor)  단어
    for videocount in np.array(os.listdir(os.path.join(DATA_PATH,kor))).astype(int): #단어 몇번 찍었는지 
        #print(videocount)    
        for currentframe in range(1):
            file=os.listdir(DATA_PATH+kor+'\\'+str(videocount)+'\\'+str(currentframe)+'\\')[0]  #마지막 파일 이름 추출
            res=np.load(os.path.join(DATA_PATH,kor,str(videocount),str(currentframe),file))  
            print(res)

[ 0.503986    0.18016624 -0.81438512 ...  0.          0.
  0.        ]
[ 0.50017273  0.21588434 -0.70597994 ...  0.          0.
  0.        ]
[ 0.48142391  0.20460632 -0.70167565 ...  0.          0.
  0.        ]
[ 0.51277357  0.22247465 -0.61003816 ...  0.          0.
  0.        ]
[ 0.505211    0.23827617 -0.5746243  ...  0.          0.
  0.        ]
[ 0.51673055  0.29977262 -0.75752544 ...  0.          0.
  0.        ]
[ 0.47986573  0.18669666 -0.65456891 ...  0.          0.
  0.        ]
[ 0.51528472  0.21945657 -0.701612   ...  0.          0.
  0.        ]
[ 0.50002778  0.19425248 -0.77658188 ...  0.          0.
  0.        ]
[ 0.51902515  0.19758998 -0.70882404 ...  0.          0.
  0.        ]
[ 0.46890163  0.21172841 -0.65786326 ...  0.          0.
  0.        ]
[ 0.5080415   0.22926794 -0.61183518 ...  0.          0.
  0.        ]
[ 0.50768751  0.22926314 -0.57964462 ...  0.          0.
  0.        ]
[ 0.4948535   0.30798835 -0.74479657 ...  0.          0.
  0.        ]
[ 0.47

[ 0.51196796  0.23968779 -0.61679387 ...  0.          0.
  0.        ]
[ 0.5329746   0.22228517 -0.4605841  ...  0.          0.
  0.        ]
[ 0.48740211  0.30253488 -0.60793191 ...  0.          0.
  0.        ]
[ 0.45585093  0.13146338 -0.55987275 ...  0.          0.
  0.        ]
[ 0.52403247  0.20920373 -0.7044009  ...  0.          0.
  0.        ]
[ 0.51007235  0.19073743 -0.69414651 ...  0.          0.
  0.        ]
[ 0.5262922  0.2019389 -0.6832124 ...  0.         0.         0.       ]
[ 0.47851571  0.22693485 -0.63008678 ...  0.          0.
  0.        ]
[ 0.50478202  0.2292738  -0.63329297 ...  0.          0.
  0.        ]
[ 0.53520554  0.22753446 -0.57750034 ...  0.          0.
  0.        ]
[ 0.48518521  0.29649547 -0.67805088 ...  0.          0.
  0.        ]
[ 0.45762366  0.13254352 -0.618393   ...  0.          0.
  0.        ]
[ 0.52686    0.20466   -0.7013942 ...  0.         0.         0.       ]
[ 0.50957638  0.19463907 -0.72240168 ...  0.          0.
  0.        ]
[ 0.

[ 0.50867093  0.19294293 -0.73024786 ...  0.          0.
  0.        ]
[ 0.51827127  0.19925444 -0.66975558 ...  0.          0.
  0.        ]
[ 0.48113522  0.22097516 -0.62327033 ...  0.          0.
  0.        ]
[ 0.51057947  0.22850403 -0.62200952 ...  0.          0.
  0.        ]
[ 0.53274983  0.2264865  -0.57999802 ...  0.          0.
  0.        ]
[ 0.49576491  0.31453341 -0.7255069  ...  0.          0.
  0.        ]
[ 0.47772121  0.13362852 -0.56513882 ...  0.          0.
  0.        ]
[ 0.52740943  0.20695053 -0.74247253 ...  0.          0.
  0.        ]
[ 0.50580436  0.18110341 -0.73944342 ...  0.          0.
  0.        ]
[ 0.51544362  0.21180122 -0.63934559 ...  0.          0.
  0.        ]
[ 0.47066849  0.22043163 -0.59088075 ...  0.          0.
  0.        ]
[ 0.51419508  0.23650256 -0.62106597 ...  0.          0.
  0.        ]
[ 0.50323188  0.22343685 -0.50969392 ...  0.          0.
  0.        ]
[ 0.49275035  0.30799854 -0.68782812 ...  0.          0.
  0.        ]
[ 0.48

[ 0.52888608  0.20429848 -0.7088443  ...  0.          0.
  0.        ]
[ 0.49765125  0.19128312 -0.75201529 ...  0.          0.
  0.        ]
[ 0.52189171  0.19461727 -0.72461945 ...  0.          0.
  0.        ]
[ 0.48468465  0.21974944 -0.57462215 ...  0.          0.
  0.        ]
[ 0.51572287  0.23184456 -0.63589776 ...  0.          0.
  0.        ]
[ 0.52813137  0.21938495 -0.64327443 ...  0.          0.
  0.        ]
[ 0.49743778  0.30759835 -0.63444239 ...  0.          0.
  0.        ]
[ 0.44358963  0.15901017 -0.77703393 ...  0.          0.
  0.        ]
[ 0.52038372  0.20259976 -0.69523096 ...  0.          0.
  0.        ]
[ 0.50458694  0.19821815 -0.7234742  ...  0.          0.
  0.        ]
[ 0.52431816  0.2060869  -0.6855917  ...  0.          0.
  0.        ]
[ 0.48273286  0.21194038 -0.49528465 ...  0.          0.
  0.        ]
[ 0.51083243  0.23506269 -0.63240582 ...  0.          0.
  0.        ]
[ 0.52907014  0.22716154 -0.73676264 ...  0.          0.
  0.        ]
[ 0.49

[ 0.5377326   0.22153717 -0.70813233 ...  0.          0.
  0.        ]
[ 0.49321857  0.30603293 -0.63592583 ...  0.          0.
  0.        ]
[ 0.45269403  0.15278389 -0.76107967 ...  0.          0.
  0.        ]
[ 0.5026893   0.19710881 -0.75067323 ...  0.          0.
  0.        ]
[ 0.51972139  0.20104298 -0.64024401 ...  0.          0.
  0.        ]
[ 0.4797793   0.22455093 -0.6024887  ...  0.          0.
  0.        ]
[ 0.50510901  0.23213682 -0.61642605 ...  0.          0.
  0.        ]
[ 0.53508222  0.23043893 -0.72867936 ...  0.          0.
  0.        ]
[ 0.49293527  0.3036657  -0.62118876 ...  0.          0.
  0.        ]
[ 0.4643181   0.13222919 -0.7878772  ...  0.          0.
  0.        ]
[ 0.49740434  0.19116265 -0.74552143 ...  0.          0.
  0.        ]
[ 0.51741636  0.19931833 -0.66385978 ...  0.          0.
  0.        ]
[ 0.47173515  0.22864348 -0.62377918 ...  0.          0.
  0.        ]
[ 0.49989161  0.23553833 -0.61676317 ...  0.          0.
  0.        ]
[ 0.53

...

[ 0.52223271  0.22813933 -0.69413126 ...  0.          0.
  0.        ]
[ 0.4942418   0.30779916 -0.63667905 ...  0.          0.
  0.        ]
[ 0.44683918  0.12095371 -0.47979951 ...  0.          0.
  0.        ]
[ 0.5117048   0.21732561 -0.80797875 ...  0.          0.
  0.        ]
[ 0.50597143  0.19666611 -0.69172561 ...  0.          0.
  0.        ]
[ 0.48546091  0.22759874 -0.68954962 ...  0.          0.
  0.        ]
[ 0.51544571  0.22392584 -0.64010173 ...  0.          0.
  0.        ]
[ 0.5208534   0.22071569 -0.54222769 ...  0.          0.
  0.        ]
[ 0.49085021  0.32967901 -0.67729521 ...  0.          0.
  0.        ]
[ 0.45337528  0.13002296 -0.51884127 ...  0.          0.
  0.        ]
[ 0.50692236  0.21802294 -0.81601858 ...  0.          0.
  0.        ]
[ 0.50501269  0.20769954 -0.71000171 ...  0.          0.
  0.        ]
[ 0.48587194  0.22584377 -0.60350013 ...  0.          0.
  0.        ]
[ 0.5077914   0.22216506 -0.63335788 ...  0.          0.
  0.        ]
[ 0.51

[ 0.51404113  0.30560228 -0.70850432 ...  0.          0.
  0.        ]
[ 0.43220097  0.13304004 -0.53253186 ...  0.          0.
  0.        ]
[ 0.5089246   0.22214739 -0.86669874 ...  0.          0.
  0.        ]
[ 0.52424806  0.1959216  -0.66636014 ...  0.          0.
  0.        ]
[ 0.50472504  0.21982726 -0.61233693 ...  0.          0.
  0.        ]
[ 0.50692213  0.23433961 -0.60680228 ...  0.          0.
  0.        ]
[ 0.52128011  0.2241185  -0.64774609 ...  0.          0.
  0.        ]
[ 0.50905722  0.31347072 -0.72962779 ...  0.          0.
  0.        ]
[ 0.44437104  0.14095998 -0.59744698 ...  0.          0.
  0.        ]
[ 0.51392621  0.21570471 -0.75006193 ...  0.          0.
  0.        ]
[ 0.5277251   0.20295158 -0.66815615 ...  0.          0.
  0.        ]
[ 0.49863696  0.23206785 -0.65594465 ...  0.          0.
  0.        ]
[ 0.51098162  0.22680362 -0.57688421 ...  0.          0.
  0.        ]
[ 0.51850659  0.22040306 -0.66279644 ...  0.          0.
  0.        ]
[ 0.51

[ 0.52762002  0.19724946 -0.73259407 ...  0.          0.
  0.        ]
[ 0.48975438  0.22919767 -0.67217278 ...  0.          0.
  0.        ]
[ 0.50159502  0.24096631 -0.57748729 ...  0.          0.
  0.        ]
[ 0.51743591  0.21125704 -0.69616681 ...  0.          0.
  0.        ]
[ 0.49974638  0.31077492 -0.65682757 ...  0.          0.
  0.        ]
[ 0.44769183  0.14869963 -0.54893076 ...  0.          0.
  0.        ]
[ 0.50010288  0.21875767 -0.81093585 ...  0.          0.
  0.        ]
[ 0.52246481  0.20395124 -0.75378048 ...  0.          0.
  0.        ]
[ 0.49687034  0.23475762 -0.6196689  ...  0.          0.
  0.        ]
[ 0.51226556  0.23877104 -0.60996187 ...  0.          0.
  0.        ]
[ 0.51897693  0.23256703 -0.66316968 ...  0.          0.
  0.        ]
[ 0.49398312  0.3065452  -0.6676625  ...  0.          0.
  0.        ]
[ 0.45144421  0.13848031 -0.52400625 ...  0.          0.
  0.        ]
[ 0.49904424  0.21993753 -0.7621361  ...  0.          0.
  0.        ]
[ 0.53

[ 0.49666342  0.24369168 -0.58794266 ...  0.          0.
  0.        ]
[ 0.53300452  0.23712826 -0.65645182 ...  0.          0.
  0.        ]
[ 0.49731126  0.30336389 -0.55301106 ...  0.          0.
  0.        ]
[ 0.45227942  0.14381568 -0.55603027 ...  0.          0.
  0.        ]
[ 0.50720274  0.22019355 -0.85330313 ...  0.          0.
  0.        ]
[ 0.51621687  0.20727931 -0.70018828 ...  0.          0.
  0.        ]
[ 0.47651801  0.22776268 -0.64998037 ...  0.          0.
  0.        ]
[ 0.50270802  0.22988328 -0.56246293 ...  0.          0.
  0.        ]
[ 0.53304702  0.23736577 -0.72529876 ...  0.          0.
  0.        ]
[ 0.49205509  0.31038129 -0.66952574 ...  0.          0.
  0.        ]
[ 0.44836336  0.12288666 -0.42587781 ...  0.          0.
  0.        ]
[ 0.51224589  0.22222844 -0.81226557 ...  0.          0.
  0.        ]
[ 0.51069808  0.21430041 -0.6989364  ...  0.          0.
  0.        ]
[ 0.47877377  0.23744544 -0.6244188  ...  0.          0.
  0.        ]
[ 0.50

KeyboardInterrupt: 

In [115]:
sequences, labels = [], []
DATA_PATH  =  'C:\\data\\train2\\'
for kor in list_aw['한국어']:
    for videocount in np.array(os.listdir(os.path.join(DATA_PATH, kor))).astype(int): #단어 안에 있는 폴더의 이름을 리스트로 출력하고 int형으로 변경 
        window = []
        for currentframe in range(432):
            # 데이터 파이썬 파일로 불러오기 
            file = os.listdir(DATA_PATH + kor + '\\'+str(videocount)+ "\\" + str(currentframe) + '\\')[0]
            res = np.load(os.path.join(DATA_PATH, kor, str(videocount),str(currentframe),file))
            # 리스트에 append 하기 
            window.append(res)
            #sequence라는 리스트안에 window가 리스트 형터로 존재 
            #리스트 안의 리스트 
        sequences.append(window)
        labels.append(label_map[kor])

In [119]:
#print(type(sequences))  #list
print(type(labels))  #list

<class 'list'>


In [117]:
import pickle
def save_variable(v,filename):
    f=open(filename,'wb')
    pickle.dump(v,f)
    f.close()
    return filename
filename=save_variable(sequences, 'E:\\sequence100.npy')

In [121]:
def load_variavle(filename):
    f=open(filename,'rb')
    r=pickle.load(f)
    f.close()
    return r
X=load_variavle('E:\\sequence100.npy')

In [120]:
#다 돌리면 sequences type확인하고 
import numpy as np
x = np.array(labels)
# 배열을 저장하기
np.save('E:/label.npy', x)

In [122]:
np.array(X).shape  #(709, 432, 1662)    709개 비디오 , 432개 프레임, 1662는 키포인트
# 총 709개의 비디오, 1편 당 30시퀀스 /  # (1662,)키포인트 -> 468*3 + 33*4 + 21*3 + 21*3 각(xyz 즉 세개씩 곱한다.)

(709, 432, 1662)

In [123]:
np.array(labels).shape  #(90,) 라벨링한 정답 데이터  #(709,)

(709,)

In [124]:
x=np.array(X)

In [125]:
# 원핫인코딩 하기 
# one hot encoding 
y = to_categorical(labels).astype(int)

In [126]:
# 데이터 나누기
# split data 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.05) # 5%가 테스트 되겠다

In [128]:
print(X_train.shape)   #(673, 432, 1662)
print(X_test.shape)    #(36, 432, 1662)
print(y_train.shape)   #(673, 100)
print(y_test.shape)   #(36, 100)

(673, 432, 1662)
(36, 432, 1662)
(673, 100)
(36, 100)


# LSTM Neural Network 만들기 

Build and Train LSTM Neural Network

In [ ]:
# 모델 생성 -> cnn 넣었을 때, 넣지 않았을 때 모두 해보기 

In [129]:
# 모델 임포트 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense  #LSTM layer 를  쓰겠다.  
from tensorflow.keras.callbacks import TensorBoard

In [130]:
# set call back ( prevent over fitting )
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
# 이 타입 쓰는 이유 CNN 과 비교 시 장점 
# 미디아 type + LSTM 레이어
# 1. Less DaTa Required  
# 2. Faster To train
# 3. Faster Detections 

In [131]:
print(list_aw['한국어'].shape[0])

100


In [135]:
model = Sequential()  # 시작 
# 30, 1662) 각각의 비디오 30 시퀀스 1662 키포인트
#######
# cnn 모델 추가 
########
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(432,1662))) #64 유닛 -> LSTM은 다음 레이어에 sequences가 필요해서 다음 레이어가 있으면 True를 해준다
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))# 다음은 dense슬라이드라 sequences 필요없어
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(list_aw['한국어'].shape[0], activation='softmax')) # ctions.shape[0] -> 3네트워크 유닛 

In [137]:
# 아담 그레이드 , 비아너리 -> 바이너리 크로스 / 이건 분류하는 거-> multi class categoty  예측하는 확률을 보여준다. 
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [138]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
22/22 [==============================] - 404s 18s/step - loss: nan - categorical_accuracy: 0.0142
Epoch 2/2000
22/22 [==============================] - 389s 17s/step - loss: nan - categorical_accuracy: 0.0091
Epoch 3/2000
22/22 [==============================] - 374s 17s/step - loss: nan - categorical_accuracy: 0.0122
Epoch 4/2000
22/22 [==============================] - 414s 19s/step - loss: nan - categorical_accuracy: 0.0156
Epoch 5/2000
22/22 [==============================] - 453s 21s/step - loss: nan - categorical_accuracy: 0.0123
Epoch 6/2000
22/22 [==============================] - 442s 20s/step - loss: nan - categorical_accuracy: 0.0201
Epoch 7/2000
22/22 [==============================] - 495s 23s/step - loss: nan - categorical_accuracy: 0.0129
Epoch 8/2000
22/22 [==============================] - 560s 25s/step - loss: nan - categorical_accuracy: 0.0134
Epoch 9/2000
22/22 [==============================] - 461s 21s/step - loss: nan - categorical_accuracy: 0.0102
E

ResourceExhaustedError:  OOM when allocating tensor with shape[32,432,1662] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node GatherV2}}]]
	 [[IteratorGetNext]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_9944]

Function call stack:
train_function


In [ ]:
model.summary()

 -  결과보기 
 - visualize the result of model



 cmd 창을 킨다 <br>
open cmd  <br>
<br>
C:\Users\User>cd Logs <br>
<br>
C:\Users\User\Logs>cd Train<br>
<br>
C:\Users\User\Logs\train>dir<br>

- 파일을 텐서플로우로 실행시킨다. (run the file )
C:\Users\User\Logs\train>tensorboard --logdir=. 

- 링크를 복사하여 크롬창에 띄운다 ( copy the like and open it at Chrome)

# 예측하기 
8. Make Predictions

In [ ]:
res = model.predict(X_test)

In [ ]:
actions[np.argmax(res[4])]

In [ ]:
actions[np.argmax(y_test[4])]

# 모델 저장 
9. Save Weights

In [ ]:
model.save('action.h5')

In [ ]:
model.load_weights('action.h5')

# 모델 평가 
10. Evaluation using Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_train)

In [ ]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)
# 이원교차표 출력 

In [ ]:
accuracy_score(ytrue, yhat)

# 실시간 예측하기 
11. Test in Real Time

In [5]:
# 동영상에 점 찍기 

def draw_landmarks(image, results):  # 동영상의 frame과 동영상에서 받은 result를 받는다. 
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # 얼굴의 점을 연결하여 그린다.
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # POSE를 연결햐여 그린다.
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # 왼손 그리기
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # 오른손 그리기 

In [6]:
# 더 정확한 판단을 위해 각각의 노드에 두개의 점을 추가한다. 
#add aditional 2 spots to the image

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),  
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
from scipy import stats

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
# 1. New detection variables #30개 차면 detect 할 수 있게
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:] #마지막 30프레임
        
        if len(sequence) == 30: #30개차면 predict한다 
            res = model.predict(np.expand_dims(sequence, axis=0))[0] # 2차원을 3차원으로 늘려준다. [0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()